# Chapter 5 — Tool Use: Google Search

The simplest ADK example — a single agent with the built-in `google_search` tool.

**Great starting point to verify your setup works.**

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Create an Agent with Google Search

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

search_agent = Agent(
    name="search_agent",
    model="gemini-2.0-flash",
    description="An agent that can search the web using Google Search.",
    instruction="You are a helpful research assistant. Use Google Search to find up-to-date information and answer the user's questions accurately. Always cite your sources.",
    tools=[google_search],
)

print("Search agent ready with tools:", [t.__name__ if hasattr(t, '__name__') else str(t) for t in search_agent.tools])

## Run the Agent

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=search_agent, app_name="search_demo", session_service=session_service)

session = await session_service.create_session(app_name="search_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="What are the latest developments in quantum computing in 2025?")]
)

response = await runner.run_async(user_id="user1", session_id=session.id, new_message=request)

for event in response:
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

## Key Takeaways

- **`google_search`** is a built-in ADK tool — no extra setup needed
- **Core pattern**: `Agent(name, model, tools) → Runner → runner.run_async()`
- The agent decides **when** to use the search tool based on the user's question